<a href="https://colab.research.google.com/github/train-hong/metro-json/blob/main/%E5%A4%A7%E6%95%B8%E6%93%9A%E6%87%89%E7%94%A8%E5%B0%88%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
stas = requests.get("https://raw.githubusercontent.com/train-hong/metro-json/main/TRTC.json").json() #站名來源

sta = {} #key = 路線, value = {車站編號: 站名}

for i in stas:
  sta[i["LineNo"]] = {}
  for j in i["Stations"]:
    sta[i["LineNo"]][j["Sequence"]] = [j["Station"]["StationID"], j["StationName"]["Zh_tw"]]

print(sta)

'''
r_sta = (stas.json())["result"]["results"] #從站名來源存取資料
r = {} #key = 流水號, value = [站名, 車站編號] 的字典
for i in range(26):
  if i < 19:
    r[i] = [r_sta[i]["stationA"], "R"+str(28-i)]
  else:
    r[i] = [r_sta[i]["stationA"], "R0"+str(28-i)] #編號數字部分為個位數時，加0
r[26] = ['捷運象山站', 'R02']

g_sta = (stas.json())["result"]["results"] #同上段
g = {}
for i in range(18):
  if i < 8:
    g[i] = [g_sta[i+43]["stationA"], "G"+str(19-i)]
  else:
    g[i] = [g_sta[i+43]["stationA"], "G0"+str(19-i)]
g[18] = ['捷運新店站', 'G01']

bl_sta = (stas.json())["result"]["results"] #同上段
bl = {}
for i in range(22):
  if i < 12:
    bl[i] = [bl_sta[i+74]["stationA"], "BL"+str(23-i)]
  else:
    bl[i] = [bl_sta[i+74]["stationA"], "BL0"+str(23-i)]
bl[22] = ['捷運頂埔站', 'BL01']

o_sta = (stas.json())["result"]["results"] #同上段
o = {}
for i in range(5):
  o[i] = [o_sta[i+137]["stationA"], "O"+str(54-i)]
for i in range(20):
  if i < 10:
    o[i+5] = [o_sta[i+153]["stationA"], "O"+str(21-i)]
  else:
    o[i+5] = [o_sta[i+153]["stationA"], "O0"+str(21-i)]
o[25] = ['捷運南勢角站', 'O01']
'''

{'BL': {'BL01': '頂埔', 'BL02': '永寧', 'BL03': '土城', 'BL04': '海山', 'BL05': '亞東醫院', 'BL06': '府中', 'BL07': '板橋', 'BL08': '新埔', 'BL09': '江子翠', 'BL10': '龍山寺', 'BL11': '西門', 'BL12': '台北車站', 'BL13': '善導寺', 'BL14': '忠孝新生', 'BL15': '忠孝復興', 'BL16': '忠孝敦化', 'BL17': '國父紀念館', 'BL18': '市政府', 'BL19': '永春', 'BL20': '後山埤', 'BL21': '昆陽', 'BL22': '南港', 'BL23': '南港展覽館'}, 'BR': {'BR01': '動物園', 'BR02': '木柵', 'BR03': '萬芳社區', 'BR04': '萬芳醫院', 'BR05': '辛亥', 'BR06': '麟光', 'BR07': '六張犁', 'BR08': '科技大樓', 'BR09': '大安', 'BR10': '忠孝復興', 'BR11': '南京復興', 'BR12': '中山國中', 'BR13': '松山機場', 'BR14': '大直', 'BR15': '劍南路', 'BR16': '西湖', 'BR17': '港墘', 'BR18': '文德', 'BR19': '內湖', 'BR20': '大湖公園', 'BR21': '葫洲', 'BR22': '東湖', 'BR23': '南港軟體園區', 'BR24': '南港展覽館'}, 'G': {'G01': '新店', 'G02': '新店區公所', 'G03': '七張', 'G04': '大坪林', 'G05': '景美', 'G06': '萬隆', 'G07': '公館', 'G08': '台電大樓', 'G09': '古亭', 'G10': '中正紀念堂', 'G11': '小南門', 'G12': '西門', 'G13': '北門', 'G14': '中山', 'G15': '松江南京', 'G16': '南京復興', 'G17': '台北小巨蛋', 'G18': '南京三民', 'G19': '松山', 'G03A'

'\nr_sta = (stas.json())["result"]["results"] #從站名來源存取資料\nr = {} #key = 流水號, value = [站名, 車站編號] 的字典\nfor i in range(26):\n  if i < 19:\n    r[i] = [r_sta[i]["stationA"], "R"+str(28-i)]\n  else:\n    r[i] = [r_sta[i]["stationA"], "R0"+str(28-i)] #編號數字部分為個位數時，加0\nr[26] = [\'捷運象山站\', \'R02\']\n\ng_sta = (stas.json())["result"]["results"] #同上段\ng = {}\nfor i in range(18):\n  if i < 8:\n    g[i] = [g_sta[i+43]["stationA"], "G"+str(19-i)]\n  else:\n    g[i] = [g_sta[i+43]["stationA"], "G0"+str(19-i)]\ng[18] = [\'捷運新店站\', \'G01\']\n\nbl_sta = (stas.json())["result"]["results"] #同上段\nbl = {}\nfor i in range(22):\n  if i < 12:\n    bl[i] = [bl_sta[i+74]["stationA"], "BL"+str(23-i)]\n  else:\n    bl[i] = [bl_sta[i+74]["stationA"], "BL0"+str(23-i)]\nbl[22] = [\'捷運頂埔站\', \'BL01\']\n\no_sta = (stas.json())["result"]["results"] #同上段\no = {}\nfor i in range(5):\n  o[i] = [o_sta[i+137]["stationA"], "O"+str(54-i)]\nfor i in range(20):\n  if i < 10:\n    o[i+5] = [o_sta[i+153]["stationA"], "O"+str(21-i)

In [14]:
from datetime import datetime
import pytz
import os
w = datetime.now(pytz.timezone('ROC')).strftime("%w") #現在星期幾

print("1 淡水信義線\n2 松山新店線\n3 板南線\n4 中和新蘆線")
line = int(input("Please enter your line number: "))
print('') #換行
st = [] #車站時刻表(station timetable)

ln = ["R", "G", "BL", "O"] #linenumber

tt = requests.get(f"https://raw.githubusercontent.com/ericyu/TaipeiMetroTimeTable/master/Lines/{ln[line-1]}.json") #時刻表(timetable)



if line == 1:
  print("1 淡水\n2 象山")
  rdi = int(input("Please enter your direction number: ")) #red line direction
  print('')
  for i in sta:
    print(i)
  #for i in range(27): #印站名列表
  #  print(i+1, r[i][0])
  sta = int(input("Please enter your station number: "))
  print('')  
  if sta == 1 and rdi == 1:
    print("你就已經在淡水了，幹嘛還要到淡水")
    os._exit(1)
  elif sta == 27 and rdi == 2:
    print("你就已經在象山了，幹嘛還要到象山")
    os._exit(1)
  #第一種寫法（因為不是每台車都有經過石牌站，所以會out of range）

  '''
  if int(w) in range(1,6): #星期幾轉換為要引用的時刻表(weekday quote)
    wq = 0
  elif w == 6:
    wq = 1
  else:
    wq = 2
  st = [] #車站時刻表(station timetable)
  tr = len((tt.json())[int(rdi)-1]["Timetables"][wq]["Trains"]) #那天有幾班車(trains)
  for i in range(tr):
    tdt = (tt.json())[int(rdi)-1]["Timetables"][wq]["Trains"][i]["Schedule"][27-sta]["DepTime"] #每班列車在該站的進站時間(train departure time)
    st.append(tdt)
  print(st)
  '''

  #第二種寫法（窮舉法，有307輛車，每輛車都停25站左右，程式要跑4分鐘才跑得完）
  '''
  if int(w) in range(1,6): #星期幾轉換為要引用的時刻表(weekday quote)
    wq = 0
  elif w == 6:
    wq = 1
  else:
    wq = 2
  tr = len((tt.json())[int(rdi)-1]["Timetables"][wq]["Trains"]) #那天有幾班車(trains)
  st = [] #車站時刻表(station timetable)
  for i in range(tr):
    sn = r[sta-1][1] #該站編號(station number)
    stp = len((tt.json())[int(rdi)-1]["Timetables"][wq]["Trains"][i]["Schedule"]) #該車經過幾站(the number of stations the train passes)
    for j in range(stp):
      if sn in (tt.json())[int(rdi)-1]["Timetables"][wq]["Trains"][i]["Schedule"][j].values():
        tdt = (tt.json())[int(rdi)-1]["Timetables"][wq]["Trains"][i]["Schedule"][j]["DepTime"] #每班列車在該站的進站時間(train departure time)
        st.append(tdt)
  '''

  #第三種寫法（第二種改良版，直接告訴程式哪站要對哪班列車的哪一項搜尋，不用每台列車每站都跑一遍）
  if rdi == 1: #往淡水
    if int(w) in range(1, 6): #星期幾轉換為要引用的時刻表(weekday quote)
      wq = 0 #週一～週五
      if sta in range(25, 28): #信義安和～象山
        for i in range(201):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][27-sta]["DepTime"])
      elif sta in range(20, 25): #台大醫院～大安
        for i in range(201):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][27-sta]["DepTime"])
        for i in range(201, 292):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][24-sta]["DepTime"])
      elif sta in range(15, 20): #台北車站～圓山
        for i in range(201):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][27-sta]["DepTime"])
        for i in range(201, 292):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][24-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][292]["Schedule"][19-sta]["DepTime"])
      elif sta in range(10, 15): #劍潭～石牌
        for i in range(201):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][27-sta]["DepTime"])
        for i in range(201, 292):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][24-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][292]["Schedule"][19-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][293]["Schedule"][14-sta]["DepTime"])
      elif sta in range(8, 10): #唭哩岸～奇岩
        for i in range(201):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][27-sta]["DepTime"])
        for i in range(201, 292):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][24-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][292]["Schedule"][19-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][293]["Schedule"][14-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][294]["Schedule"][9-sta]["DepTime"])
      elif sta in range(6, 8): #北投～復興崗
        for i in range(51, 201):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][27-sta]["DepTime"])
        for i in range(288, 292):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][24-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][292]["Schedule"][19-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][293]["Schedule"][14-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][294]["Schedule"][9-sta]["DepTime"])
      elif sta in range(3, 6): #忠義～竹圍
        for i in range(51, 201):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][27-sta]["DepTime"])
        for i in range(288, 292):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][24-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][292]["Schedule"][19-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][293]["Schedule"][14-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][294]["Schedule"][9-sta]["DepTime"])
        for i in range(295, 306):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][5-sta]["DepTime"])
      elif sta == 2: #紅樹林      
        for i in range(51, 201):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][27-sta]["DepTime"])
        for i in range(288, 292):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][24-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][292]["Schedule"][19-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][293]["Schedule"][14-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][294]["Schedule"][9-sta]["DepTime"])
        for i in range(295, 306):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][5-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][306]["Schedule"][2-sta]["DepTime"])
    else:
      wq = 1 #週六、週日（兩個時刻表一樣）
      if sta in range(25, 28): #信義安和～象山
        for i in range(124):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][27-sta]["DepTime"])
      elif sta in range(20, 25): #台大醫院～大安
        for i in range(124):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][27-sta]["DepTime"])
        for i in range(124, 230):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][24-sta]["DepTime"])
      elif sta in range(15, 20): #台北車站～圓山
        for i in range(124):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][27-sta]["DepTime"])
        for i in range(124, 230):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][24-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][230]["Schedule"][19-sta]["DepTime"])
      elif sta in range(10, 15): #劍潭～石牌
        for i in range(124):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][27-sta]["DepTime"])
        for i in range(124, 230):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][24-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][230]["Schedule"][19-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][231]["Schedule"][14-sta]["DepTime"])
      elif sta in range(8, 10): #唭哩岸～奇岩
        for i in range(124):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][27-sta]["DepTime"])
        for i in range(124, 230):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][24-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][230]["Schedule"][19-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][231]["Schedule"][14-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][232]["Schedule"][9-sta]["DepTime"])
      elif sta in range(6, 8): #北投～復興崗
        for i in range(3, 124):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][27-sta]["DepTime"])
        for i in range(226, 230):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][24-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][230]["Schedule"][19-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][231]["Schedule"][14-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][232]["Schedule"][9-sta]["DepTime"])
      elif sta in range(3, 6): #忠義～竹圍
        for i in range(3, 124):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][27-sta]["DepTime"])
        for i in range(226, 230):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][24-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][230]["Schedule"][19-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][231]["Schedule"][14-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][232]["Schedule"][9-sta]["DepTime"])
        for i in range(233, 237):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][5-sta]["DepTime"])
      elif sta == 2: #紅樹林      
        for i in range(3, 124):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][27-sta]["DepTime"])
        for i in range(226, 230):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][24-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][230]["Schedule"][19-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][231]["Schedule"][14-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][232]["Schedule"][9-sta]["DepTime"])
        for i in range(233, 237):
          st.append((tt.json())[0]["Timetables"][wq]["Trains"][i]["Schedule"][5-sta]["DepTime"])
        st.append((tt.json())[0]["Timetables"][wq]["Trains"][237]["Schedule"][2-sta]["DepTime"])
  else: #往象山
    if int(w) in range(1, 6): #星期幾轉換為要引用的時刻表(weekday quote)
      wq = 0 #週一～週五
      if sta in range(1, 5): #淡水～關渡
        for i in range(150):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-1]["DepTime"])
      elif sta in range(5, 7): #忠義、復興崗
        for i in range(150):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-1]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][150]["Schedule"][sta-5]["DepTime"])
      elif sta in range(7, 9): #北投、奇岩
        for i in range(150):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-1]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][150]["Schedule"][sta-5]["DepTime"])
        for i in range(151, 287):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-7]["DepTime"])
      elif sta == 9: #唭哩岸
        for i in range(150):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-1]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][150]["Schedule"][sta-5]["DepTime"])
        for i in range(151, 287):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-7]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][287]["Schedule"][sta-9]["DepTime"])
      elif sta in range(10, 14): #石牌～士林
        for i in range(150):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-1]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][150]["Schedule"][sta-5]["DepTime"])
        for i in range(151, 287):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-7]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][287]["Schedule"][sta-9]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][288]["Schedule"][sta-10]["DepTime"])
      elif sta in range(14, 16): #劍潭、圓山
        for i in range(150):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-1]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][150]["Schedule"][sta-5]["DepTime"])
        for i in range(151, 287):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-7]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][287]["Schedule"][sta-9]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][288]["Schedule"][sta-10]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][289]["Schedule"][sta-14]["DepTime"])
      elif sta in range(16, 19): #民權西路～中山
        for i in range(150):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-1]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][150]["Schedule"][sta-5]["DepTime"])
        for i in range(151, 287):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-7]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][287]["Schedule"][sta-9]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][288]["Schedule"][sta-10]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][289]["Schedule"][sta-14]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][290]["Schedule"][sta-16]["DepTime"])
      elif sta in range(19, 24): #台北車站～大安森林公園
        for i in range(150):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-1]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][150]["Schedule"][sta-5]["DepTime"])
        for i in range(151, 287):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-7]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][287]["Schedule"][sta-9]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][288]["Schedule"][sta-10]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][289]["Schedule"][sta-14]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][290]["Schedule"][sta-16]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][291]["Schedule"][sta-19]["DepTime"])
      elif sta in range(24, 27): #大安～台北101/世貿
        for i in range(150):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-1]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][150]["Schedule"][sta-5]["DepTime"])
        for i in range(151, 199):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-7]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][287]["Schedule"][sta-9]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][288]["Schedule"][sta-10]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][289]["Schedule"][sta-14]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][290]["Schedule"][sta-16]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][291]["Schedule"][sta-19]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][292]["Schedule"][sta-24]["DepTime"])
    else: #週六、週日（兩個時刻表一樣）
      wq = 1
      if sta in range(1, 5): #淡水～關渡
        for i in range(120):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-1]["DepTime"])
      elif sta in range(5, 7): #忠義、復興崗
        for i in range(120):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-1]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][120]["Schedule"][sta-5]["DepTime"])
      elif sta in range(7, 9): #北投、奇岩
        for i in range(120):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-1]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][120]["Schedule"][sta-5]["DepTime"])
        for i in range(121, 225):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-7]["DepTime"])
      elif sta == 9: #唭哩岸
        for i in range(120):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-1]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][120]["Schedule"][sta-5]["DepTime"])
        for i in range(121, 225):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-7]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][225]["Schedule"][sta-9]["DepTime"])
      elif sta in range(10, 14): #石牌～士林
        for i in range(120):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-1]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][120]["Schedule"][sta-5]["DepTime"])
        for i in range(121, 225):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-7]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][225]["Schedule"][sta-9]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][226]["Schedule"][sta-10]["DepTime"])
      elif sta in range(14, 16): #劍潭、圓山
        for i in range(120):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-1]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][120]["Schedule"][sta-5]["DepTime"])
        for i in range(121, 225):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-7]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][225]["Schedule"][sta-9]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][226]["Schedule"][sta-10]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][227]["Schedule"][sta-14]["DepTime"])
      elif sta in range(16, 19): #民權西路～中山
        for i in range(120):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-1]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][120]["Schedule"][sta-5]["DepTime"])
        for i in range(121, 225):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-7]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][225]["Schedule"][sta-9]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][226]["Schedule"][sta-10]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][227]["Schedule"][sta-14]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][228]["Schedule"][sta-16]["DepTime"])
      elif sta in range(19, 24): #台北車站～大安森林公園
        for i in range(120):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-1]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][120]["Schedule"][sta-5]["DepTime"])
        for i in range(121, 225):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-7]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][225]["Schedule"][sta-9]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][226]["Schedule"][sta-10]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][227]["Schedule"][sta-14]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][228]["Schedule"][sta-16]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][229]["Schedule"][sta-19]["DepTime"])
      elif sta in range(24, 27): #大安～台北101/世貿
        for i in range(120):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-1]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][120]["Schedule"][sta-5]["DepTime"])
        for i in range(121, 225):
          st.append((tt.json())[1]["Timetables"][wq]["Trains"][i]["Schedule"][sta-7]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][225]["Schedule"][sta-9]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][226]["Schedule"][sta-10]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][227]["Schedule"][sta-14]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][228]["Schedule"][sta-16]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][229]["Schedule"][sta-19]["DepTime"])
        st.append((tt.json())[1]["Timetables"][wq]["Trains"][230]["Schedule"][sta-24]["DepTime"])

#
#
#

elif line == 2:
  print("1 松山\n2 新店")
  gdi = input("Please enter your direction number: ") #green line direction
  print('')
  for i in range(19): #印站名列表
    print(i+1, g[i][0])
  sta = int(input("Please enter your station number: "))
  tt = requests.get("https://raw.githubusercontent.com/ericyu/TaipeiMetroTimeTable/master/Lines/G.json") #時刻表(timetable)
  if sta == 1 and rdi == 1:
    print("你就已經在松山了，幹嘛還要到松山")
  elif sta == 19 and rdi == 2:
    print("你就已經在新店了，幹嘛還要到新店")
  
'''
elif line == 3:
  print("1 南港展覽館\n2 頂埔")
  direc = input("Please enter your direction number: ")
  print('')
  for i in range(23):
    print(i+1, bl[i][0])
  timetable = requests.get("https://raw.githubusercontent.com/ericyu/TaipeiMetroTimeTable/master/Lines/BL.json") #時刻表
elif line == 4:
  print("1 迴龍\n2 蘆洲\n3 南勢角")
  direc = input("Please enter your direction number: ")
  print('')
  for i in range(26):
    print(i+1, o[i][0])
  timetable = requests.get("https://raw.githubusercontent.com/ericyu/TaipeiMetroTimeTable/master/Lines/O.json") #時刻表

'''


h = int(datetime.now(pytz.timezone('ROC')).strftime("%-H")) #現在幾點（小時）
m = int(datetime.now(pytz.timezone('ROC')).strftime("%-M")) #現在幾點（分鐘）
tn = h*60 + m #time now 

jt = [] #時間轉為該天第幾分鐘（純時間數字）(just time)
for i in range(len(st)):
  jt.append((int(st[i][0])*10 + int(st[i][1])) * 60 + int(st[i][3])*10 + int(st[i][4]))
jt.append(tn)
jt.sort()

'''
#二分搜尋法
def binary_search(jt, low, high, tn):
  if high >= low:
    mid = (high = low) // 2
    if jt[mid] == tn:
      return 0
    elif jt[mid] < tn:
      return binary_search(arr, mid + 1, high, x)
    else:
      return binary_search(arr, low, mid - 1, x)
  else:
    return jt[mid] - tn
'''

tnp = jt.index(tn) #time now position

if tn > jt[-1]:
  tn = tn-1440
'''
if jt[tnp+1] == tn:
  at = 0 #arrive time
else:
  at = jt[tnp+1] - tn
'''

print('捷運淡水站往象山方向列車 距離下次發車還有3分鐘')
print(tn, jt)


1 淡水信義線
2 松山新店線
3 板南線
4 中和新蘆線

1 淡水
2 象山

BL
BR
G
O
R
Y


KeyboardInterrupt: ignored

In [ ]:
11111111121

294


週一～五<br>
往北投 21站 象山～北投 0-50 51台車<br>
往淡水 27站 象山～淡水 51-200 150台車<br>
往北投 18站 大安～北投 201-287 87台車<br>
往淡水 24站 大安～淡水 288-291 4台車<br>
往淡水 19站 台北車站～淡水 292 1台車<br>
往淡水 14站 劍潭～淡水 293 1台車<br>
往淡水 9站 唭哩岸～淡水 294 1台車<br>
往淡水 5站 忠義～淡水 295-305 11台車<br>
往淡水 2站 紅樹林～淡水 306 1台車<br><br>
週六<br>
往北投 21站 象山～北投 0-2<br>
往淡水 27站 象山～淡水 3-123<br>
往北投 18站 大安～北投 124-225<br>
往淡水 24站 大安～淡水 226-229<br>
往淡水 19站 台北車站～淡水 230<br>
往淡水 14站 劍潭～淡水 231<br>
往淡水 9站 唭哩岸～淡水 232<br>
往淡水 5站 忠義～淡水 233-236<br>
往淡水 2站 紅樹林～淡水 237<br><br>
週日<br>
往北投 21站 象山～北投 0-2<br>
往淡水 27站 象山～淡水 3-123<br>
往北投 18站 大安～北投 124-225<br>
往淡水 24站 大安～淡水 226-229<br>
往淡水 19站 台北車站～淡水 230<br>
往淡水 14站 劍潭～淡水 231<br>
往淡水 9站 唭哩岸～淡水 232<br>
往淡水 5站 忠義～淡水 233-236<br>
往淡水 2站 紅樹林～淡水 237<br><br>